# Recommendation Algorithms
This script demonstrates and tests recommendation algorithms on [Movie Lens Dataset](https://grouplens.org/datasets/movielens/).

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Check and prepare MovieLens dataset.

MovieLens provides one [small](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) (1MB) and one [large](https://files.grouplens.org/datasets/movielens/ml-latest.zip) (25MB) dataset, which describes 5-star rating and free-text tagging activity.

- **ml-1m**: 100836 ratings, 3683 tag applications across 9742 movies, 610 users
- **ml-25m**: 25000095 ratings, 1093360 tag applications across 62423 movies, 162541 users

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files genome-scores.csv, genome-tags.csv, links.csv, movies.csv, ratings.csv and tags.csv.

In [2]:
if not os.path.isdir('data'):
    os.mkdir('data')

data_root = 'data/ml-1m'
zip_file = data_root+'.zip'

if not os.path.isdir(data_root):
    if not os.path.exists(zip_file):
        import requests
        response = requests.get("https://files.grouplens.org/datasets/movielens/ml-latest-small.zip")
        open(zip_file, "wb").write(response.content)

    import zipfile
    with zipfile.ZipFile(zip_file, 'r') as ref:
        ref.extractall('data')  # root directory
        os.rename('data/ml-latest-small', data_root)
    os.remove(zip_file)

print('Dataset was downloaded.')

Dataset was downloaded.


In [3]:
movies = pd.read_csv(os.path.join(data_root, 'movies.csv'))
rating = pd.read_csv(os.path.join(data_root, 'ratings.csv'))
links = pd.read_csv(os.path.join(data_root, 'links.csv'))
tags = pd.read_csv(os.path.join(data_root, 'tags.csv'))

# check number of users and movies
user_num = len(rating.userId.unique())
movie_num = len(movies)
print(f'Movie number: {movie_num}, user number: {user_num}')

movies['newMovieId'] = movies.index
new_rating = rating.merge(movies, on='movieId', how='left')
new_rating['newUserId'] = new_rating['userId']-1

movies.head(2)

Movie number: 9742, user number: 610


,movieId,title,genres,newMovieId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1


In [4]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [5]:
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


In [6]:
new_rating.head(2)

,userId,movieId,rating,timestamp,title,genres,newMovieId,newUserId
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,2,0


#### 1.1. User-Item dataset
The target is to get a matrix. Rows -> Movies. Columns -> Users.

In [7]:
len(rating['movieId'].unique())

9724

In [8]:
utility_matrix = np.empty([movie_num, user_num])
utility_matrix[:] = np.nan
utility_matrix[new_rating['newMovieId'], new_rating['newUserId']] = new_rating['rating']
mask = np.isnan(utility_matrix)
utility_matrix, utility_matrix.shape

(array([[4. , nan, nan, ..., 2.5, 3. , 5. ],
        [nan, nan, nan, ..., 2. , nan, nan],
        [4. , nan, nan, ..., 2. , nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 (9742, 610))

Calculate the mean and standard deviation of rating for each movie. Since users do not rate each movie, there must be some missing values in the utility matrix. When we normalize the data, we should thus mask the unrated values out.

In [9]:
masked_arr = np.ma.masked_array(utility_matrix, mask)
rating_mean = masked_arr.mean(axis=1)
rating_mean = rating_mean.data[:,np.newaxis]

rating_std = masked_arr.std(axis=1)+1e-6
rating_std = rating_std.data[:,np.newaxis]

masked_arr = (masked_arr-rating_mean)/rating_std
masked_arr = masked_arr.filled(0)
masked_arr

array([[ 0.09493122,  0.        ,  0.        , ..., -1.70596992,
        -1.10566954,  1.29553199],
       [ 0.        ,  0.        ,  0.        , ..., -1.63133437,
         0.        ,  0.        ],
       [ 0.70875169,  0.        ,  0.        , ..., -1.20579833,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### 2.1. Singular Value Decomposition (SVD)

<center><img src="https://latex.codecogs.com/svg.image?M=U\Sigma&space;V^H" title="https://latex.codecogs.com/svg.image?M=U\Sigma V^H" /><center/>

奇异值分解又称谱分解，是将矩阵分解成以上形式，U和V是酉矩阵(unitary matrix)特征向量。目的是将数据谱分解后计算相似性。

- M (mxn) 为待分析的矩阵数据，如图片或关系矩阵
- H为共轭转置(Hermitian operator)
- U (mxm) 组成一套对M的正交"输入"的基向量
- V (nxn) 组成一套对M的正交"输出"的基向量
- 基向量相互正交，类似于傅里叶变换中的正弦基的概念
- Σ (mxn) 是特征对角矩阵，对角线上的值为奇异值

奇异值往往对应着矩阵中隐含的重要信息，且重要性和奇异值大小正相关。奇异向量是对某一信息的描述，而奇异值则衡量了这些信息对于M的权重。

如对图片数据奇异值分解，则只要保留topN个奇异值，就可以有损恢复原图，因为后面的几个奇异值对应的可能是噪声。

[爱因斯坦求和范式 Einstein summation convention](https://www.cnblogs.com/dechinphy/p/einsum-examples.html)可以实现点乘、元素乘、求和等操作。

Explanation of SVD for the case of MovieLens
- m: movie number, n: user number
- M: m x n
- U: from movies to latent feature (m x r)
- V: from latent feature to users (r x n)
- Σ: latent feature

Calculate similarity between movies
- truncate U to top-N features
- get latent features from U by indexing
- calculate cosine similarity between features

In [10]:
U, S, V = np.linalg.svd(masked_arr)
k = 50
truncated_features = U[:, :k]

In [19]:
query_movie = 'Jumanji (1995)'    # id=1
queried_movie_index = movies.where(movies['title'] == query_movie).first_valid_index()

queried_movie_feature = truncated_features[queried_movie_index, :]
# 'ij, ij -> i' equal to np.sum(np.square(truncated_features), axis=1)
magnitude = np.sqrt(np.einsum('ij, ij -> i', truncated_features, truncated_features))

top_n = 10
similarity = np.dot(truncated_features, queried_movie_feature.T) / ((magnitude[queried_movie_index] * magnitude)+1e-6)
sort_indexes = np.argsort(-similarity)[:top_n]
print(len(sort_indexes))

# movies.loc[new_movie_index]['newMovieId']
movies.loc[sort_indexes]
# list(movies['title'].to_numpy()[sort_indexes])


10


,movieId,title,genres,newMovieId
1,2,Jumanji (1995),Adventure|Children|Fantasy,1
126,153,Batman Forever (1995),Action|Adventure|Comedy|Crime,126
1845,2454,"Fly, The (1958)",Horror|Mystery|Sci-Fi,1845
6853,62113,How to Lose Friends & Alienate People (2008),Comedy,6853
140,168,First Knight (1995),Action|Drama|Romance,140
253,292,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,253
146,174,Jury Duty (1995),Comedy,146
8835,132046,Tomorrowland (2015),Action|Adventure|Children|Mystery|Sci-Fi,8835
1968,2613,Night of the Comet (1984),Comedy|Horror|Sci-Fi,1968
1728,2322,Soldier (1998),Action|Sci-Fi|War,1728
